<a href="https://colab.research.google.com/github/danoff/lending-a-hand-in-bringing-ballcarriers-down/blob/main/Run_regressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Check the versions of libraries

# Python version
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))

import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from scipy import stats
from scipy.stats import pearsonr
from statsmodels.compat import lzip
import statsmodels
import matplotlib.pyplot as plt



%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score
from sklearn import metrics

%matplotlib inline
import seaborn as sns

# Load libraries

from pandas.plotting import scatter_matrix
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
scipy: 1.11.4
numpy: 1.25.2
matplotlib: 3.7.1
pandas: 2.0.3
sklearn: 1.2.2


In [29]:
# read CSV of analytics staff data

data = pd.read_csv('speed_dist_data_20231228.csv', header=0)
data = data.dropna()

In [30]:
# Look at the variable names

variablenames = data.keys()
print (variablenames)

Index(['gameId', 'playId', 'frameId', 'playType', 'ballCarrierNflId',
       'ballCarrierDisplayName', 'ballCarrierJerseyNumber', 'ballCarrierSpeed',
       'toPlayerNflId', 'toPlayerDisplayName', 'toPlayerDistanceToBallCarrier',
       'toPlayerAction_tackle', 'toPlayerAction_assist',
       'toPlayerAction_forcedFumble', 'toPlayerAction_missedTackle'],
      dtype='object')


In [31]:
# describe csv of our derived NextGenStats data

data.describe()

,gameId,playId,frameId,ballCarrierNflId,ballCarrierJerseyNumber,ballCarrierSpeed,toPlayerNflId,toPlayerDistanceToBallCarrier,toPlayerAction_tackle,toPlayerAction_assist,toPlayerAction_forcedFumble,toPlayerAction_missedTackle
count,5.828669e+06,5.828669e+06,5.828669e+06,5.828669e+06,5.828669e+06,5.828669e+06,5.828669e+06,5.828669e+06,5.828669e+06,5.828669e+06,5.828669e+06,5.828669e+06
mean,2.022099e+09,1.981037e+03,2.551169e+01,4.821888e+04,2.658427e+01,3.722866e+00,4.745870e+04,1.136076e+01,7.001581e-02,4.777746e-02,5.973920e-04,2.093171e-02
std,5.933397e+03,1.170471e+03,1.765263e+01,4.592537e+03,2.131973e+01,2.537812e+00,4.997624e+03,8.707313e+00,2.551737e-01,2.132951e-01,2.443430e-02,1.431558e-01
min,2.022091e+09,5.400000e+01,1.000000e+00,2.551100e+04,1.000000e+00,0.000000e+00,3.313100e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.022093e+09,9.490000e+02,1.100000e+01,4.487900e+04,1.200000e+01,1.530000e+00,4.345500e+04,4.774620e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.022101e+09,1.997000e+03,2.300000e+01,4.780700e+04,2.400000e+01,3.700000e+00,4.652100e+04,9.287310e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.022102e+09,2.979000e+03,3.700000e+01,5.294200e+04,3.100000e+01,5.640000e+00,5.254700e+04,1.633590e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,2.022111e+09,5.096000e+03,1.640000e+02,5.515800e+04,8.900000e+01,1.080000e+01,5.524100e+04,9.675010e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [32]:
# show headers from csv of our derived NextGenStats data

data.head()

,gameId,playId,frameId,playType,ballCarrierNflId,ballCarrierDisplayName,ballCarrierJerseyNumber,ballCarrierSpeed,toPlayerNflId,toPlayerDisplayName,toPlayerDistanceToBallCarrier,toPlayerAction_tackle,toPlayerAction_assist,toPlayerAction_forcedFumble,toPlayerAction_missedTackle
1,2022090800,56,1,Completion,42489,Stefon Diggs,14,6.23,38577,Bobby Wagner,11.01360,0,0,0,0
2,2022090800,56,1,Completion,42489,Stefon Diggs,14,6.23,41239,Aaron Donald,12.42790,0,0,0,0
3,2022090800,56,1,Completion,42489,Stefon Diggs,14,6.23,42816,Troy Hill,31.57940,0,0,0,0
4,2022090800,56,1,Completion,42489,Stefon Diggs,14,6.23,43294,Jalen Ramsey,3.63629,1,0,0,0
5,2022090800,56,1,Completion,42489,Stefon Diggs,14,6.23,43298,Leonard Floyd,12.25040,0,0,0,0


In [33]:
# Get the number of rows
row_count = len(data)

print(f'The number of rows in the CSV file is: {row_count}')

The number of rows in the CSV file is: 5828669


In [34]:
# made a dataframe to do some maniupulation
df = data

In [35]:
# removed the football which has a PlayerNflId of zero
df = df[df['toPlayerNflId'] != 0]

In [43]:
#describe dataframe table

row_count2 = len(df)

print(f'The number of rows in the CSV file is: {row_count2}')

The number of rows in the CSV file is: 283336


In [37]:
# only included players who were lss than one yard away from the ballcarrier

df = df[df['toPlayerDistanceToBallCarrier'] < 1]

In [38]:
# Regular regression to ballcarrier speed based on defender distance

model = smf.ols("ballCarrierSpeed ~ toPlayerDistanceToBallCarrier", data= df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       ballCarrierSpeed   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     2409.
Date:                Sat, 06 Jul 2024   Prob (F-statistic):               0.00
Time:                        22:41:46   Log-Likelihood:            -5.7921e+05
No. Observations:              283336   AIC:                         1.158e+06
Df Residuals:                  283334   BIC:                         1.158e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         2.0978      0.010    206.056      0.000       2.078       2.118
toPlayerDistanceToBallCarrier     0.7537      0.015     49.084      0.000       0.724       0.784
==============================================================================
Omnibus:                    29450.509   Durbin-Watson:                   0.206
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            39463.075
Skew:                           0.896   Prob(JB):                         0.00
Kurtosis:                       3.360   Cond. No.                         6.13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
corr_coefficient, p_value = pearsonr(df['ballCarrierSpeed'], df['toPlayerDistanceToBallCarrier'])

print(f'The correlation between ballCarrierSpeed and toPlayerDistanceToBallCarrier is: {corr_coefficient}')
print(f'The p-value is: {p_value}')

The correlation between ballCarrierSpeed and toPlayerDistanceToBallCarrier is: 0.09182238125616067
The p-value is: 0.0


In [40]:
# on second dataframe removed the football which has a PlayerNflId of zero
df2 = data
df2 = df2[df2['toPlayerNflId'] != 0]

In [41]:
# only included players who were less than two yards away from the ballcarrier

df2 = df2[df2['toPlayerDistanceToBallCarrier'] < 2]

In [42]:
# Get the number of rows
row_count = len(df2)

print(f'The number of rows in the CSV file is: {row_count}')

The number of rows in the CSV file is: 602516


In [44]:
# Regular regression to ballcarrier speed based on defender distance for those under 2 yards

model2 = smf.ols("ballCarrierSpeed ~ toPlayerDistanceToBallCarrier", data= df2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       ballCarrierSpeed   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                 1.818e+04
Date:                Sat, 06 Jul 2024   Prob (F-statistic):               0.00
Time:                        22:43:41   Log-Likelihood:            -1.2905e+06
No. Observations:              602516   AIC:                         2.581e+06
Df Residuals:                  602514   BIC:                         2.581e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         2.1227      0.006    338.608      0.000       2.110       2.135
toPlayerDistanceToBallCarrier     0.7099      0.005    134.849      0.000       0.700       0.720
==============================================================================
Omnibus:                    41448.136   Durbin-Watson:                   0.127
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            50625.301
Skew:                           0.709   Prob(JB):                         0.00
Kurtosis:                       2.913   Cond. No.                         4.58
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [45]:
corr_coefficient2, p_value = pearsonr(df2['ballCarrierSpeed'], df2['toPlayerDistanceToBallCarrier'])

print(f'The correlation between ballCarrierSpeed and toPlayerDistanceToBallCarrier is: {corr_coefficient2}')
print(f'The p-value is: {p_value}')

The correlation between ballCarrierSpeed and toPlayerDistanceToBallCarrier is: 0.1711618977753881
The p-value is: 0.0


In [46]:
# on third  dataframe removed the football which has a PlayerNflId of zero
df3 = data
df3 = df3[df3['toPlayerNflId'] != 0]

# only included players who were less than 3 yards away from the ballcarrier

df3 = df3[df3['toPlayerDistanceToBallCarrier'] < 3]

# Regular regression to ballcarrier speed based on defender distance for those under 3 yards

model3 = smf.ols("ballCarrierSpeed ~ toPlayerDistanceToBallCarrier", data= df3).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       ballCarrierSpeed   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                 4.740e+04
Date:                Sat, 06 Jul 2024   Prob (F-statistic):               0.00
Time:                        22:45:12   Log-Likelihood:            -1.9768e+06
No. Observations:              907680   AIC:                         3.954e+06
Df Residuals:                  907678   BIC:                         3.954e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         2.2246      0.005    455.795      0.000       2.215       2.234
toPlayerDistanceToBallCarrier     0.6063      0.003    217.719      0.000       0.601       0.612
==============================================================================
Omnibus:                    44896.805   Durbin-Watson:                   0.130
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            48061.378
Skew:                           0.538   Prob(JB):                         0.00
Kurtosis:                       2.662   Cond. No.                         4.85
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [47]:
corr_coefficient3, p_value = pearsonr(df3['ballCarrierSpeed'], df3['toPlayerDistanceToBallCarrier'])

print(f'The correlation between ballCarrierSpeed and toPlayerDistanceToBallCarrier is: {corr_coefficient3}')
print(f'The p-value is: {p_value}')

The correlation between ballCarrierSpeed and toPlayerDistanceToBallCarrier is: 0.22278044756735838
The p-value is: 0.0


In [48]:
# on fourth  dataframe removed the football which has a PlayerNflId of zero
df4 = data
df4 = df4[df4['toPlayerNflId'] != 0]

# only included players who were less than 3 yards away from the ballcarrier

df4 = df4[df4['toPlayerDistanceToBallCarrier'] < 4]

# Regular regression to ballcarrier speed based on defender distance for those under 2 yards

model4 = smf.ols("ballCarrierSpeed ~ toPlayerDistanceToBallCarrier", data= df4).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       ballCarrierSpeed   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                 8.144e+04
Date:                Sat, 06 Jul 2024   Prob (F-statistic):               0.00
Time:                        22:45:44   Log-Likelihood:            -2.6657e+06
No. Observations:             1217656   AIC:                         5.331e+06
Df Residuals:                 1217654   BIC:                         5.331e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         2.3507      0.004    569.377      0.000       2.343       2.359
toPlayerDistanceToBallCarrier     0.5056      0.002    285.382      0.000       0.502       0.509
==============================================================================
Omnibus:                    51070.686   Durbin-Watson:                   0.140
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            47259.621
Skew:                           0.429   Prob(JB):                         0.00
Kurtosis:                       2.558   Cond. No.                         5.64
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:

# Compute the Pearson correlation coefficient and p-value
# thanks ChatGPT for the help


corr_coefficient4, p_value = pearsonr(df4['ballCarrierSpeed'], df4['toPlayerDistanceToBallCarrier'])

print(f'The correlation between ballCarrierSpeed and toPlayerDistanceToBallCarrier is: {corr_coefficient4}')
print(f'The p-value is: {p_value}')

The correlation between ballCarrierSpeed and toPlayerDistanceToBallCarrier is: 0.25038375761242126
The p-value is: 0.0
